# Classification of redistribution

AUC-ROC analysis to obtain the threshold which provided minimal False Positive Rate.

In [ ]:
## logging functions
from icecream import ic as info
import logging
## system functions
from os.path import dirname
import sys
## system functions from roux
from roux.lib.io import read_table
from roux.lib.io import to_dict
from roux.lib.io import to_table
## data functions from roux
import roux.lib.dfs as rd # attributes
sys.path.append('../')

In [2]:
## parameters
input_path=None
output_path=None # f'{output_dir_path}/01_classified.tsv'
method_cutoff=None

In [4]:
output_dir_path=dirname(output_path)
logging.info(output_dir_path)

## Scores

In [ ]:
## redistribution score
df01=read_table(input_path,drop_index=True,)
## validate no missing values
assert not df01['redistribution score'].isnull().any()
df01.head(1)

## Binarisation

In [7]:
if method_cutoff.lower()=='gmm':
    from roux.stat.cluster import cluster_1d
    cutoff=cluster_1d(
        df01.log.query(expr='`control` != False')['redistribution score'].astype(float),
        n_clusters=2,
        random_state=1,
        returns=['coff'],
        test=True)['coff']
    info(cutoff)
elif method_cutoff.lower() in ['roc','pr']:
    ## redistribution score with the controls
    df1=df01.log.dropna(subset=['control'])
    from roux.stat.binary import get_cutoff
    d1=get_cutoff(
        y_true=df1['control'].values,
        y_score=df1['redistribution score'].values,
        method=method_cutoff.lower(),
        show_cutoff=dict(maximize='specificity',),
        returns=['data','ax','cutoff'],
        )
    d1['data']=d1['data'].melt(id_vars=['threshold'])    
    to_table(d1['data'],
        f"{output_dir_path}/01_classified/curve.tsv",
           )
    to_dict(
        d1['cutoff'].to_dict(),
        f"{output_dir_path}/stats.yaml",
    )
    cutoff=d1['cutoff']['threshold']
info(cutoff)

### Apply threshold

In [ ]:
df01=(df01
    .assign(
    **{
        'redistribution': lambda df : df['redistribution score']>=cutoff,
    })
)
df01['redistribution'].value_counts()

### Save Table

In [ ]:
to_table(df01,
         output_path,
        )
to_dict(
    {'cutoff':cutoff},
    f'{output_dir_path}/01_classified/stats.json',
)